# <h1>Projet IA premetent d'analyser tes extrait de film et de les reconnaitre.

**Nous avons emit un porbleme que nous allons essaye de resoudre avec cette IA:**
<br><br>
On ces rendus compte que dans notre vie de tout les jours ils arrivent apres moment avoir le besion de connaitre le non d'un flime d'ont nous avons entendue un extrait car il nous samble sympa a regardé
<br>
On ces donc donner pour objectif de crée un Shazme qui analyse un extrait d'un film ou d'une série, et qui nous donne comme information le film, l'auteur, les acteur.texte en italique
<br><br>
Notre IA vas utilise de un Aprrentisage superviser car nous allons luis donnée des jeux de donnée affain qu'elle retourne le titre du film, nous somme dans un type de probleme de régréssion, et nous allons détremine avec un arbre de désistion 

Suite a de nombreuse recherche et de questionnement je me suis rendue compte que il n'exister pas de dataset qui répertorier tout les film avec des anotatoin qui permet de crée un model qui pourrai faire que l'on peut analyser que le fichier audio sois reconnue.


Et il serai beaucoup trop long de crée notre propre base de donné avec tout les film anoter.


<h1> Création du jeux de donnée pour l'apprentisage


In [3]:
!git clone https://github.com/ElclemclemGithub/Action

Cloning into 'Action'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
path_audio= "BDD film IA/"

In [8]:
import librosa
import numpy as np
from tensorflow.keras.models import load_model

# Charger un fichier audio
audio_file = 'file.mp3'
y, sr = librosa.load(audio_file, sr=44100)

# Extraire des caractéristiques audio
# Par exemple, extraire les coefficients MFCC
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

# Mise en forme des caractéristiques audio pour le modèle
mfccs = np.expand_dims(mfccs, axis=-1)
mfccs = np.expand_dims(mfccs, axis=0)

# Charger le modèle de reconnaissance de musique de film entraîné
model = load_model('path/to/your/model.h5')

# Prédire la musique de film
prediction = model.predict(mfccs)
movie = np.argmax(prediction)

# Afficher le titre du film
if movie == 0:
    print("La musique provient du film 'Inception'")
elif movie == 1:
    print("La musique provient du film 'Interstellar'")
elif movie == 2:
    print("La musique provient du film 'The Dark Knight'")

OSError: ignored

In [9]:
import librosa
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Charger les données audio et extraire les caractéristiques
def extract_features(file_path):
    audio_data, sr = librosa.load(file_path, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    return mfccs_scaled

# Charger les données et les préparer pour l'entraînement du modèle
def load_data(audio_dir):
    features = []
    labels = []
    for root, dirs, files in os.walk(audio_dir):
        for file in files:
            file_path = os.path.join(root, file)
            label = root.split('/')[-1]
            try:
                feature = extract_features(file_path)
                features.append(feature)
                labels.append(label)
            except Exception as e:
                print("Error encountered while processing {} : {}".format(file_path, e))
                continue
    # Encoder les labels en valeurs numériques
    encoder = LabelEncoder()
    labels = encoder.fit_transform(labels)
    # Normaliser les caractéristiques
    scaler = StandardScaler()
    features = scaler.fit_transform(np.array(features, dtype=np.float32))
    # Séparer les données en ensemble d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

# Entraîner un modèle de forêt aléatoire
def train_model(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy: {:.2f}%".format(accuracy * 100))
    return clf

# Charger les données audio et entraîner le modèle
audio_dir = 'BDD film IA/'
X_train, X_test, y_train, y_test = load_data(audio_dir)
model = train_model(X_train, X_test, y_train, y_test)


ValueError: ignored

In [10]:
import librosa
import numpy as np
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras.models import load_model

# Chargement des extraits audio
def load_data(dataset_path):
    X = []
    y = []
    for folder in os.listdir(dataset_path):
        sub_folder = os.path.join(dataset_path, folder)
        for filename in os.listdir(sub_folder):
            file_path = os.path.join(sub_folder, filename)
            try:
                # Chargement de l'extrait audio
                audio_data, sr = librosa.load(file_path, res_type='kaiser_fast')
                # Extrait les features audio avec MFCC
                mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=40)
                # Redimensionne les features pour avoir la même longueur
                pad_width = 174 - mfccs.shape[1]
                mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
                # Ajoute les features et le label dans les listes X et y
                X.append(mfccs)
                y.append(folder)
            except Exception as e:
                print(f"Error loading audio file {file_path}: {e}")
    return np.array(X), np.array(y)

# Charger les données et diviser en ensembles d'entraînement et de test
X, y = load_data('chemin/vers/votre/dataset')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir les labels en vecteurs binaires
num_labels = len(np.unique(y))
y_train = to_categorical(y_train, num_labels)
y_test = to_categorical(y_test, num_labels)

# Création du modèle
model = Sequential()
model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels))
model.add(Activation('softmax'))

# Compilation du modèle
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam())

# Définir le rappel de sauvegarde de modèle
checkpoint = ModelCheckpoint(filepath='chemin/vers/votre/modèle', verbose=1, save_best_only=True)

# Entraînement du modèle
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test), callbacks=[checkpoint])

# Evaluation du modèle sur l'ensemble de test
loss, acc = model.evaluate(X_test, y_test, batch_size=32)
print(f"Test accuracy: {acc}")

def predict(file_path, model_path):
    # Charger le modèle
    model = load_model(model_path)
    # Charger l'extrait audio
    audio_data, sr = librosa.load(file_path, res_type='kaiser_fast')
    # Extraire les features audio
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=40)
    # Redimensionner les features pour avoir la même longueur
    pad_width = 174 - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    # Faire une prédiction sur les features
    prediction = model.predict(np.expand_dims(mfccs, axis=0))
    # Récupérer le label prédit
    predicted_label = np.argmax(prediction, axis=1)
    return predicted_label

FileNotFoundError: ignored